In [1]:
import copy
import math

# 外皮情報LV3からLV4へのコンバート

## 概要

外皮性能計算プログラムに、外皮の用途別情報を追加したものより、内壁等（建物内部の壁、天井、床）の面積の推定および熱貫流率の決定を行う。  
- 内壁等の面積は、室の周長を求める際の係数（アスペクト比）と階高を仮定して、外皮の用途別情報および用途別床面積より推定する。
- 内壁等の熱貫流率は一般的な仕様を想定して与える（石膏ボード12.5mm－空気層－石膏ボード12.5mm、参考：SimHeatでの熱負荷計算用の無断熱の層構成）。室内戸は考慮しない。
- その他は、内壁の用途別情報が得られている場合と同様に求める。

## 1. Common Element

'Common'要素はそのまま維持する。

参照：InputDataDefinition.ipynb

##### Function

In [2]:
def convert_common(d):
    return {
            'Region'                     : d['Common']['Region'],
            'IsSimplifiedInput'          : d['Common']['IsSimplifiedInput'],
            'MainOccupantRoomFloorArea'  : d['Common']['MainOccupantRoomFloorArea'],
            'OtherOccupantRoomFloorArea' : d['Common']['OtherOccupantRoomFloorArea'],
            'TotalFloorArea'             : d['Common']['TotalFloorArea']
    }

## 2. Walls Element

'Walls'要素はそのまま維持する。

##### Function

In [3]:
def make_wall( d_wall ):
    
    wall = copy.deepcopy(d_wall)
    
    return wall

In [4]:
def convert_wall( d ):    
    return [ make_wall(f) for f in d['Walls'] ]

## 3. Windows Element

'Windows'要素はそのまま維持する。

##### Function

In [5]:
def make_window( d_window ):
    
    window = copy.deepcopy(d_window)
    
    return window

In [6]:
def convert_window( d ):    
    return [ make_window(f) for f in d['Windows'] ]

## 4. Doors Element

'Doors'要素はそのまま維持する。

##### Function

In [7]:
def make_door( d_door ):

    door = copy.deepcopy(d_door)
    
    return door

In [8]:
def convert_door( d ):    
    return [ make_door(f) for f in d['Doors'] ]

## 5. Heatbridges Element

'Heatbridges'要素はそのまま維持する。

##### Function

In [9]:
def make_heatbridge( d_heatbridge ):

    heatbridge = copy.deepcopy(d_heatbridge)

    return heatbridge

In [10]:
def convert_heatbridge( d ):    
    return [ make_heatbridge(f) for f in d['Heatbridges'] ]

## 6. EarthfloorPerimeters Element

'EarthfloorPerimeters'要素はそのまま維持する。

##### Function

In [11]:
def make_earthfloorperimeter( d_earthfloorperimeter ):
    
    earthfloorperimeter = copy.deepcopy(d_earthfloorperimeter)

    return earthfloorperimeter

In [12]:
def convert_earthfloorperimeter( d ):    
    return [ make_earthfloorperimeter(f) for f in d['EarthfloorPerimeters'] ]

## 7. Earthfloors Element

'Earthfloors'要素はそのまま維持する。

##### Function

In [13]:
def make_earthfloor( d_earthfloor ):

    earthfloor = copy.deepcopy(d_earthfloor)
        
    return earthfloor

In [14]:
def convert_earthfloor( d ):        
    return [ make_earthfloor(f) for f in d['Earthfloors'] ]

## 8. InnerWalls Element

### 8.1. 共通して用いる関数

#### 1) 用途別床面積を取得する関数

In [15]:
def get_floor_area(a_mr, a_or, a_total):
    return a_mr, a_or, a_total - a_mr - a_or

#### 2) 方位が「下面」である外皮の部位の面積の合計

In [16]:
def get_floor_area_to_outside(d):

    parts = []

    if 'Walls' in d:
        parts.extend(d['Walls'])
    
    if 'Windows' in d:
        parts.extend(d['Windows'])
        
    if 'Doors' in d:
        parts.extend(d['Doors'])
    
    def f(room_type):
        return sum(part['area'] for part in parts if part['space'] == room_type and part['direction'] == 'bottom')

    return f('main'), f('other'), f('nonliving')

#### 3) 方位が「上面」である外皮の部位の面積の合計

In [17]:
def get_ceiling_area_to_outside(d):

    parts = []
    
    if 'Walls' in d:
        parts.extend(d['Walls'])
        
    if 'Windows' in d:
        parts.extend(d['Windows'])

    if 'Doors' in d:
        parts.extend(d['Doors'])
        
    def f(room_type):
        return sum(part['area'] for part in parts if part['space'] == room_type and part['direction'] == 'top')
    
    return f('main'), f('other'), f('nonliving')

#### 4) 方位が水平（上面・下面以外）である外皮の部位の面積の合計

In [18]:
def get_outside_wall_area(d):

    parts = []

    if 'Walls' in d:
        parts.extend(d['Walls'])
        
    if 'Windows' in d:
        parts.extend(d['Windows'])
        
    if 'Doors' in d:
        parts.extend(d['Doors'])

    def f(room_type):
        return sum(part['area'] for part in parts \
                   if part['space'] == room_type and part['direction'] != 'top' and part['direction'] != 'bottom')
    
    return f('Main'), f('Other'), f('Nonliving')

#### 5) 土間床等の面積の合計

In [19]:
def get_area_of_earth_floor(d):

    def f(room_type):
        return sum(part['area'] for part in d['Earthfloors'] if part['space'] == room_type)
    
    if 'Earthfloors' in d:
        return f('main'), f('other'), f('nonliving')
    else:
        return 0.0, 0.0, 0.0

#### 6) 床下空間に属する土間床等の面積の取得

In [20]:
def get_area_of_earth_floor_in_under_floor(d):
    if 'Earthfloors' in d:
        return sum(earthfloor['area'] for earthfloor in d['Earthfloors'] if earthfloor['space'] == 'underfloor')
    else:
        return 0.0

### 8.2. InnerWalls Element - Ground Floor 

#### 1) 層構成

以下の層構成とする。

| 材料 | 厚さ, m | 熱伝導率, W m<sup>-1</sup> K<sup>-1</sup> | 熱抵抗, m<sup>2</sup> k W<sup>-1</sup> | 容積比熱, J L<sup>-1</sup> K<sup>-1</sup> |
|---|---|---|---|
| 合板 | 0.012 | 0.16 | 0.075 | 720.0 |

##### Function

In [21]:
def get_inner_floor_layers_over_under_floor():
    return [{'name': 'PED', 'cond': 0.16, 'thick': 0.012, 'specH': 720.0}]

#### 2) 床下空間上の床面積の推定

- 外皮の用途別情報が入力された場合において床下空間に属する土間床等面積が0㎡より大きい場合、室と床下空間の間の床の面積の合計は、床下空間に属する土間床等面積の合計（ただし、室用途別の床面積と方位が「下面」の部位面積の差分の合計を上限とする）に等しいとみなす。  
- 室と床下空間の間の床は、室用途別の床面積と方位が「下面」の部位面積の差分に応じて、各用途に割り当てる。

$\sum_{i}^{}A_{b,i}>0$ かつ $A_{dif,MR} + A_{dif,OR} + A_{dif,NR}>0$ の場合

$$
\displaystyle
A_{f,MR}= \sum_{i}^{}A_{b,i} \times \frac{ A_{dif,MR} }{ A_{dif,MR} + A_{dif,OR} + A_{dif,NR} }
$$

$$
\displaystyle
A_{f,OR}= \sum_{i}^{}A_{b,i} \times \frac{ A_{dif,OR} }{ A_{dif,MR} + A_{dif,OR} + A_{dif,NR} }
$$

$$
\displaystyle
A_{f,NR}= \sum_{i}^{}A_{b,i} \times \frac{ A_{dif,NR} }{ A_{dif,MR} + A_{dif,OR} + A_{dif,NR} }
$$

$A_{b,i}$: 床下空間に属する土間床等の部位$i$の面積, m<sup>2</sup>  
　　 ($\sum_{i}^{}A_{b,i}≦A_{dif,MR} + A_{dif,OR} + A_{dif,NR}$を想定)  
$A_{f,MR}$: 主たる居室と床下空間の間の床の面積, m<sup>2</sup>  
$A_{f,OR}$: その他の居室と床下空間の間の床の面積, m<sup>2</sup>  
$A_{f,NR}$: 非居室と床下空間の間の床の面積, m<sup>2</sup>  
$A_f$: 室と床下空間の間の床面積の合計, m<sup>2</sup>  
$A_{dif,MR}$: 主たる居室において、方位が「下面」の部位の面積と土間床等の面積の合計と、床面積の差, m<sup>2</sup>  
$A_{dif,OR}$: その他の居室において、方位が「下面」の部位の面積と土間床等の面積の合計と、床面積の差, m<sup>2</sup>  
$A_{dif,NR}$: 非居室において、方位が「下面」の部位の面積と土間床等の面積の合計と、床面積の差, m<sup>2</sup>  

$$
\displaystyle
A_{dif, MR} = max(0, A_{MR} - \sum_i A_{of,MR,i} - \sum_i A_{ef,MR,i})
$$

$$
\displaystyle
A_{dif, OR} = max(0, A_{OR} - \sum_i A_{of,OR,i} - \sum_i A_{ef,OR,i})
$$

$$
\displaystyle
A_{dif, NR} = max(0, A_{NR} - \sum_i A_{of,NR,i} - \sum_i A_{ef,NR,i})
$$

$A_{MR}$: 主たる居室の床面積, m<sup>2</sup>  
$A_{OR}$: その他の居室の床面積, m<sup>2</sup>  
$A_{NR}$: 非居室の床面積, m<sup>2</sup>   
$A_{of,MR,i}$: 方位が「下面」である主たる居室の外皮の部位(一般部位又は開口部)$i$の面積, m<sup>2</sup>  
$A_{of,OR,i}$: 方位が「下面」であるその他の居室の外皮の部位(一般部位又は開口部)$i$の面積, m<sup>2</sup>  
$A_{of,NR,i}$: 方位が「下面」である非居室の外皮の部位(一般部位又は開口部)$i$の面積, m<sup>2</sup>  
$A_{ef,MR,i}$: 主たる居室の土間床$i$の面積, m<sup>2</sup>  
$A_{ef,OR,i}$: その他の居室の土間床$i$の面積, m<sup>2</sup>  
$A_{ef,NR,i}$: 非居室の土間床$i$の面積, m<sup>2</sup>  

##### Function

In [22]:
def get_area_floor_over_underfloor(d):
    
    # 用途別床面積の取得
    a_mr, a_or, a_nr = get_floor_area(a_mr=d['Common']['MainOccupantRoomFloorArea'],
                                      a_or=d['Common']['OtherOccupantRoomFloorArea'],
                                      a_total=d['Common']['TotalFloorArea'])
    
    # 下面（direction:Bottom）の壁・窓・ドア面積の合計値の取得
    a_outside_mr, a_outside_or, a_outside_nr = get_floor_area_to_outside(d)
    
    # 土間床等の面積の合計値の取得
    a_earth_floor_mr, a_earth_floor_or, a_earth_floor_nr = get_area_of_earth_floor(d)
    
    # 方位が「下面」の部位の面積と土間床等の面積の合計値と、床面積の差
    a_dif_mr = max(0.0, a_mr - a_outside_mr - a_earth_floor_mr)
    a_dif_or = max(0.0, a_or - a_outside_or - a_earth_floor_or)
    a_dif_nr = max(0.0, a_nr - a_outside_nr - a_earth_floor_nr)

    a_dif = a_dif_mr + a_dif_or + a_dif_nr

    # 床下空間に属する土間床等の面積の取得
    a_b = get_area_of_earth_floor_in_under_floor(d)

    # 居室と床下空間との間の床の面積
    # 室と床下空間の間の床面を設定する場合
    if a_b > 0.0 and a_dif > 0.0:
        a_f_mr = a_b * a_dif_mr / a_dif
        a_f_or = a_b * a_dif_or / a_dif
        a_f_nr = a_b * a_dif_nr / a_dif
        return a_f_mr, a_f_or, a_f_nr
    # 室と床下空間の間の床面を設定しない場合
    else :
        return 0.0, 0.0, 0.0

#### 3) 室と床下空間の間の床の構築

##### Function

In [23]:
def get_innerwall_groundfloor(d):
    
    # 室と床下空間の間の床面積の推定
    a_if_mr, a_if_or, a_if_nr = get_area_floor_over_underfloor(d)
    
    # 壁体名称、部位面積を更新し、「壁体が属する空間種類」「隣接空間種類」を追加する
    ifs = []
    
    if a_if_mr > 0.0:
        ifs.append({
            'name' : 'GroundFloor_Main',
            'type' : 'GroundFloor',
            'direction' : 'Horizontal',
            'area' : a_if_mr,
            'Layers' : get_inner_floor_layers_over_under_floor(),
            'space' : 'main',
            'nextspace': 'underfloor'
        })
        ifs.append({
            'name' : 'GroundFloor_Main',
            'type' : 'GroundFloor',
            'direction' : 'Horizontal',
            'area' : a_if_mr,
            'Layers' : get_inner_floor_layers_over_under_floor(),
            'space' : 'underfloor',
            'nextspace': 'main'
        })
    
    if a_if_or > 0.0:
        ifs.append({
            'name' : 'GroundFloor_Other',
            'type' : 'GroundFloor',
            'direction' : 'Horizontal',
            'area' : a_if_or,
            'Layers' : get_inner_floor_layers_over_under_floor(),
            'space' : 'other',
            'nextspace': 'underfloor'
        })
        ifs.append({
            'name' : 'GroundFloor_Other',
            'type' : 'GroundFloor',
            'direction' : 'Horizontal',
            'area' : a_if_or,
            'Layers' : get_inner_floor_layers_over_under_floor(),
            'space' : 'underfloor',
            'nextspace': 'other'                
        })
        
    if a_if_nr > 0.0:
        ifs.append({
            'name' : 'GroundFloor_Nonliving',
            'type' : 'GroundFloor',
            'direction' : 'Horizontal',
            'area' : a_if_nr,
            'Layers' : get_inner_floor_layers_over_under_floor(),
            'space' : 'nonliving',
            'nextspace': 'underfloor'
        })
        ifs.append({
            'name' : 'GroundFloor_Nonliving',
            'type' : 'GroundFloor',
            'direction' : 'Horizontal',
            'area' : a_if_nr,
            'Layers' : get_inner_floor_layers_over_under_floor(),
            'space' : 'underfloor',
            'nextspace': 'nonliving'
        })
        
    return ifs

##### Example

In [24]:
d = {
    'Common': {
        'Region': 6,
        'IsSimplifiedInput': False,
        'MainOccupantRoomFloorArea': 30.0,
        'OtherOccupantRoomFloorArea': 60.0,
        'TotalFloorArea': 120.0
    },
    'Walls': [
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'wood', 'InputMethod' :'InputUA', 
          'direction': 'top', 'area': 60, 'space': 'main', 'UA': 0.24, 'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'wood', 'InputMethod' :'InputAllDetails', 'direction': 'top', 
          'area': 60, 'space': 'main',
          'Parts': [{'AreaRatio': 0.8, 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                  {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'AreaRatio': 0.2, 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                  {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
          'IsSunshadeInput': False },
        { 'name': 'Floor', 'type': 'Floor', 'structure': 'wood', 'InputMethod' :'InputAllLayers', 'direction': 'bottom', 
          'area': 60, 'space': 'main',
          'FloorConstructionMethod' :'FrameInsulcolumn',
          'Parts': [{'TypeFloor': 'Insulation', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                           {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'TypeFloor': 'Heatbridge', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                           {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
           'IsSunshadeInput': False }
    ],
    'Windows': [
        { 'name': 'WindowTop', 'direction': 'top', 'area': 30, 'space': 'main', 'UW': 6.51, 'TypeWindow': 'Single', 
          'IsEtaValueInput': False, 'TypeFrame': 'WoodOrResin', 'TypeGlass': '3WgG', 'TypeShade': 'Shoji', 'IsSunshadeInput': False }
    ],
    'Earthfloors': [
        { 'name': 'other', 'area': 5.0, 'space': 'underfloor' },
        { 'name': 'entrance', 'area': 5.0, 'space': 'underfloor' }
    ]
}

innerwall_groundfloor = get_innerwall_groundfloor(d)
innerwall_groundfloor

[{'name': 'GroundFloor_Other',
  'type': 'GroundFloor',
  'direction': 'Horizontal',
  'area': 6.666666666666667,
  'Layers': [{'name': 'PED', 'cond': 0.16, 'thick': 0.012, 'specH': 720.0}],
  'space': 'other',
  'nextspace': 'underfloor'},
 {'name': 'GroundFloor_Other',
  'type': 'GroundFloor',
  'direction': 'Horizontal',
  'area': 6.666666666666667,
  'Layers': [{'name': 'PED', 'cond': 0.16, 'thick': 0.012, 'specH': 720.0}],
  'space': 'underfloor',
  'nextspace': 'other'},
 {'name': 'GroundFloor_Nonliving',
  'type': 'GroundFloor',
  'direction': 'Horizontal',
  'area': 3.3333333333333335,
  'Layers': [{'name': 'PED', 'cond': 0.16, 'thick': 0.012, 'specH': 720.0}],
  'space': 'nonliving',
  'nextspace': 'underfloor'},
 {'name': 'GroundFloor_Nonliving',
  'type': 'GroundFloor',
  'direction': 'Horizontal',
  'area': 3.3333333333333335,
  'Layers': [{'name': 'PED', 'cond': 0.16, 'thick': 0.012, 'specH': 720.0}],
  'space': 'underfloor',
  'nextspace': 'nonliving'}]

### 8.3. InnerWalls Element - Inner Floor

#### 1) 層構成

以下の層構成とする。

| 材料 | 厚さ, m | 熱伝導率, W m<sup>-1</sup> K<sup>-1</sup> | 熱抵抗, m<sup>2</sup> k W<sup>-1</sup> | 容積比熱, J L<sup>-1</sup> K<sup>-1</sup> |
|---|---|---|---|
| 合板 | 0.012 | 0.16 | 0.075 | 720.0 |

##### Function

In [25]:
def get_inner_floor_layers():
    return [{'name': 'PED', 'cond': 0.16, 'thick': 0.012, 'specH': 720.0}]

#### 2) 室上の床面積の推定

$$
\displaystyle
A_{if,MR}= \max \left(0, A_{MR} -  \left(A_{gf,MR} + \sum_i A_{of,MR,i} + \sum_i A_{ef,MR,i} \right)\right)
$$

$$
\displaystyle
A_{if,OR}= \max \left(0, A_{OR} -  \left(A_{gf,OR} + \sum_i A_{of,OR,i} + \sum_i A_{ef,OR,i} \right)\right)
$$

$$
\displaystyle
A_{if,NR}= \max \left(0, A_{NR} -  \left(A_{gf,NR} + \sum_i A_{of,NR,i} + \sum_i A_{ef,NR,i} \right)\right)
$$

$A_{if,MR}$: その他の居室または非居室との間の、主たる居室の床の面積, m<sup>2</sup>  
$A_{if,OR}$: 主たる居室または非居室との間の、その他の居室の床の面積, m<sup>2</sup>  
$A_{if,NR}$: 主たる居室またはその他の居室との間の、非居室の床の面積, m<sup>2</sup>  
$A_{MR}$: 主たる居室の床面積, m<sup>2</sup>  
$A_{OR}$: その他の居室の床面積, m<sup>2</sup>  
$A_{NR}$: 非居室の床面積, m<sup>2</sup>   
$A_{gf,MR}$: 主たる居室と床下空間の間の床の面積, m<sup>2</sup>  
$A_{gf,OR}$: その他の居室と床下空間の間の床の面積, m<sup>2</sup>  
$A_{gf,NR}$: 非居室と床下空間の間の床の面積, m<sup>2</sup>  
$A_{of,MR,i}$: 方位が「下面」である主たる居室の外皮の部位(一般部位又は開口部)$i$の面積, m<sup>2</sup>  
$A_{of,OR,i}$: 方位が「下面」であるその他の居室の外皮の部位(一般部位又は開口部)$i$の面積, m<sup>2</sup>  
$A_{of,NR,i}$: 方位が「下面」である非居室の外皮の部位(一般部位又は開口部)$i$の面積, m<sup>2</sup>  
$A_{ef,MR,i}$: 主たる居室の土間床$i$の面積, m<sup>2</sup>  
$A_{ef,OR,i}$: その他の居室の土間床$i$の面積, m<sup>2</sup>  
$A_{ef,NR,i}$: 非居室の土間床$i$の面積, m<sup>2</sup>   

##### Function

In [26]:
def get_floor_area_over_room(d, innerwalls):

    # 床下空間上の床の面積の合計
    def f(s):
        # s : space name
        return sum(iw['area'] for iw in innerwalls if (iw['type'] == 'GroundFloor' and iw['space'] == s and iw['nextspace'] == 'underfloor'))       
    
    # 用途別床面積の取得
    a_mr, a_or, a_nr = get_floor_area(a_mr=d['Common']['MainOccupantRoomFloorArea'],
                                      a_or=d['Common']['OtherOccupantRoomFloorArea'],
                                      a_total=d['Common']['TotalFloorArea'])
    
    # 床下空間上の床面積と下面（Direction:Bottom）の壁・窓・ドア面積と土間床等の面積の合計値の取得
    a_f_mr, a_f_or, a_f_nr = f('main'), f('other'), f('nonliving')
    
    # 方位が「下面」である外皮の部位の面積の合計
    a_outside_mr, a_outside_or, a_outside_nr = get_floor_area_to_outside(d)

    # 土間床の面積
    a_earth_floor_mr, a_earth_floor_or, a_earth_floor_nr = get_area_of_earth_floor(d)
    
    # 床下空間上の床面積と方位が「下面」の部位の面積と土間床等の面積の合計値と、床面積の差を、室上の床面積とみなす。
    a_if_mr = max(0.0, a_mr - a_f_mr - a_outside_mr - a_earth_floor_mr)
    a_if_or = max(0.0, a_or - a_f_or - a_outside_or - a_earth_floor_or)
    a_if_nr = max(0.0, a_nr - a_f_nr - a_outside_nr - a_earth_floor_nr)
    
    return a_if_mr, a_if_or, a_if_nr

##### Example

In [27]:
d = {
    'Common': {
        'Region': 6,
        'IsSimplifiedInput': False,
        'MainOccupantRoomFloorArea': 30.0,
        'OtherOccupantRoomFloorArea': 60.0,
        'TotalFloorArea': 120.0
    },
    'Walls': [
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'wood', 'InputMethod' :'InputUA', 
          'direction': 'top', 'area': 60, 'space': 'main', 'UA': 0.24, 'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'wood', 'InputMethod' :'InputAllDetails', 'direction': 'top', 
          'area': 60, 'space': 'main',
          'Parts': [{'AreaRatio': 0.8, 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                  {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'AreaRatio': 0.2, 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                  {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
          'IsSunshadeInput': False },
        { 'name': 'Floor', 'type': 'Floor', 'structure': 'wood', 'InputMethod' :'InputAllLayers', 'direction': 'bottom', 
          'area': 60, 'space': 'main',
          'FloorConstructionMethod' :'FrameInsulcolumn',
          'Parts': [{'TypeFloor': 'Insulation', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                           {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'TypeFloor': 'Heatbridge', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                           {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
           'IsSunshadeInput': False }
    ],
    'Windows': [
        { 'name': 'WindowTop', 'direction': 'top', 'area': 30, 'space': 'main', 'UW': 6.51, 'TypeWindow': 'Single', 
          'IsEtaValueInput': False, 'TypeFrame': 'WoodOrResin', 'TypeGlass': '3WgG', 'TypeShade': 'Shoji', 'IsSunshadeInput': False }
    ],
    'Earthfloors': [
        { 'name': 'other', 'area': 5.0, 'space': 'underfloor' },
        { 'name': 'entrance', 'area': 5.0, 'space': 'underfloor' }
    ]
}

innerwalls = [
        { 'area': 60, 'name': 'GroundFloor_Main', 'type' : 'GroundFloor', 'space': 'main', 'nextspace' : 'underfloor', 'direction' : 'Horizontal', 'Layers' : get_inner_floor_layers()},
        { 'area': 15, 'name': 'GroundFloor_Other', 'type' : 'GroundFloor', 'space': 'other', 'nextspace' : 'underfloor', 'direction' : 'Horizontal', 'Layers' : get_inner_floor_layers()},
]

get_floor_area_over_room(d, innerwalls)

(0.0, 45.0, 30.0)

#### 3) 室下の天井面積の推定

$$
\displaystyle
A_{ic,MR}= \max \left(0, A_{MR} -  \sum_i A_{oc,MR,i} \right)
$$

$$
\displaystyle
A_{ic,OR}= \max \left(0, A_{OR} -  \sum_i A_{oc,OR,i} \right)
$$

$$
\displaystyle
A_{ic,NR}= \max \left(0, A_{NR} -  \sum_i A_{oc,NR,i} \right)
$$

$A_{ic,MR}$: その他の居室または非居室との間の、主たる居室の天井の面積, m<sup>2</sup>  
$A_{ic,OR}$: 主たる居室または非居室との間の、その他の居室の天井の面積, m<sup>2</sup>  
$A_{ic,NR}$: 主たる居室またはその他の居室との間の、非居室の天井の面積, m<sup>2</sup>  
$A_{MR}$: 主たる居室の床面積, m<sup>2</sup>  
$A_{OR}$: その他の居室の床面積, m<sup>2</sup>  
$A_{NR}$: 非居室の床面積, m<sup>2</sup>   
$A_{oc,MR,i}$: 方位が「上面」である主たる居室の外皮の部位(一般部位又は開口部)$i$の面積, m<sup>2</sup>  
$A_{oc,OR,i}$: 方位が「上面」であるその他の居室の外皮の部位(一般部位又は開口部)$i$の面積, m<sup>2</sup>  
$A_{oc,NR,i}$: 方位が「上面」である非居室の外皮の部位(一般部位又は開口部)$i$の面積, m<sup>2</sup>  

##### Function

In [28]:
def get_ceiling_area_below_room(d):
    
    # 用途別床面積の取得
    a_mr, a_or, a_nr = get_floor_area(a_mr=d['Common']['MainOccupantRoomFloorArea'],
                                      a_or=d['Common']['OtherOccupantRoomFloorArea'],
                                      a_total=d['Common']['TotalFloorArea'])
    
    # 上面（Direction:Top）の壁・窓・ドア面積の合計値の取得
    a_outside_mr, a_outside_or, a_outside_nr = get_ceiling_area_to_outside(d)
    
    # 方位が「上面」の部位の面積の合計値と、床面積の差
    a_ic_mr = max(0.0, a_mr - a_outside_mr)
    a_ic_or = max(0.0, a_or - a_outside_or)
    a_ic_nr = max(0.0, a_nr - a_outside_nr)
   
    return a_ic_mr, a_ic_or, a_ic_nr

##### Example

In [29]:
d = {
    'Common': {
        'Region': 6,
        'IsSimplifiedInput': False,
        'MainOccupantRoomFloorArea': 30.0,
        'OtherOccupantRoomFloorArea': 60.0,
        'TotalFloorArea': 120.0
    },
    'Walls': [
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'wood', 'InputMethod' :'InputUA', 
          'direction': 'top', 'area': 60, 'space': 'main', 'UA': 0.24, 'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'wood', 'InputMethod' :'InputAllDetails', 'direction': 'top', 
          'area': 60, 'space': 'main',
          'Parts': [{'AreaRatio': 0.8, 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                  {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'AreaRatio': 0.2, 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                  {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
          'IsSunshadeInput': False },
        { 'name': 'Floor', 'type': 'Floor', 'structure': 'wood', 'InputMethod' :'InputAllLayers', 'direction': 'bottom', 
          'area': 60, 'space': 'main',
          'FloorConstructionMethod' :'FrameInsulcolumn',
          'Parts': [{'TypeFloor': 'Insulation', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                           {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'TypeFloor': 'Heatbridge', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                           {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
           'IsSunshadeInput': False }
    ],
    'Windows': [
        { 'name': 'WindowTop', 'direction': 'top', 'area': 30, 'space': 'main', 'UW': 6.51, 'TypeWindow': 'Single', 
          'IsEtaValueInput': False, 'TypeFrame': 'WoodOrResin', 'TypeGlass': '3WgG', 'TypeShade': 'Shoji', 'IsSunshadeInput': False }
    ],
    'Earthfloors': [
        { 'name': 'other', 'area': 5.0, 'space': 'underfloor' },
        { 'name': 'entrance', 'area': 5.0, 'space': 'underfloor' }
    ]
}

get_ceiling_area_below_room(d)

(0.0, 60.0, 30.0)

### 4) 室間の床と天井の面積の推定

$A_{if,MR}$,$A_{if,OR}$,$A_{if,NR}$は、既知とする。  
$A_{if,MR}$,$A_{if,OR}$,$A_{if,NR}$は、各用途の床下空間上の床と、方位が「下面」である外皮の部位と、土間床等の合計面積を、用途別床面積より差し引いて推定する。  
これらより、$A_{if,MR,OR}$,$A_{if,MR,NR}$,$A_{if,OR,NR}$,$A_{if,OR,NR}$,$A_{if,NR,MR}$,$A_{if,NR,OR}$を推定する。

各用途の室上の床面積は、その用途と他の2用途の間の室上の床面積の合計に等しい。

$$
\displaystyle
A_{if,MR} = A_{if,MR,OR}+A_{if,MR,NR}
$$
$$
\displaystyle
A_{if,OR} = A_{if,OR,MR}+A_{if,OR,NR}
$$
$$
\displaystyle
A_{if,NR} = A_{if,NR,MR}+A_{if,NR,OR}
$$

$A_{if,MR}$: その他の居室または非居室との間の、主たる居室の床の面積, m<sup>2</sup>  
$A_{if,OR}$: 主たる居室または非居室との間の、その他の居室の床の面積, m<sup>2</sup>  
$A_{if,NR}$: 主たる居室またはその他の居室との間の、非居室の床の面積, m<sup>2</sup>  
$A_{if,MR,OR}$: その他の居室との間の、主たる居室の床の面積, m<sup>2</sup>  
$A_{if,MR,NR}$: 非居室との間の、主たる居室の床の面積, m<sup>2</sup>  
$A_{if,OR,MR}$: 主たる居室との間の、その他の居室の床の面積, m<sup>2</sup>  
$A_{if,OR,NR}$: 非居室との間の、その他の居室の床の面積, m<sup>2</sup>  
$A_{if,NR,MR}$: 主たる居室との間の、非居室の床の面積, m<sup>2</sup>  
$A_{if,NR,OR}$: その他の居室との間の、非居室の床の面積, m<sup>2</sup>  

2用途の間の天井面積は、その2用途の間の床面積と一致するものとする。  

$$
\displaystyle
A_{ic,MR,OR} = A_{if,OR,MR}
$$
$$
\displaystyle
A_{ic,MR,NR} = A_{if,NR,MR}
$$
$$
\displaystyle
A_{ic,OR,MR} = A_{if,MR,OR}
$$
$$
\displaystyle
A_{ic,OR,NR} = A_{if,NR,OR}
$$
$$
\displaystyle
A_{ic,NR,MR} = A_{if,MR,NR}
$$
$$
\displaystyle
A_{ic,NR,OR} = A_{if,OR,NR}
$$

$A_{ic,MR,OR}$: その他の居室との間の、主たる居室の天井の面積, m<sup>2</sup>  
$A_{ic,MR,NR}$: 非居室との間の、主たる居室の天井の面積, m<sup>2</sup>  
$A_{ic,OR,MR}$: 主たる居室との間の、その他の居室の天井の面積, m<sup>2</sup>  
$A_{ic,OR,NR}$: 非居室との間の、その他の居室の天井の面積, m<sup>2</sup>  
$A_{ic,NR,MR}$: 主たる居室との間の、非居室の天井の面積, m<sup>2</sup>  
$A_{ic,NR,OR}$: その他の居室との間の、非居室の天井の面積, m<sup>2</sup>  

各用途と他の2用途との間の室下の天井面積の比率は、他の2用途の室上の床面積の比率に一致すると仮定する。

$$
\displaystyle
A_{ic,MR,OR}:A_{ic,MR,NR} = A_{if,OR,MR}:A_{if,NR,MR} = A_{if,OR}:A_{if,NR}
$$
$$
\displaystyle
A_{ic,OR,MR}:A_{ic,OR,NR} = A_{if,MR,OR}:A_{if,NR,OR} = A_{if,MR}:A_{if,NR}
$$
$$
\displaystyle
A_{ic,NR,MR}:A_{ic,NR,OR} = A_{if,MR,NR}:A_{if,OR,NR} = A_{if,MR}:A_{if,OR}
$$

以上より、$A_{if,MR,OR}$,$A_{if,MR,NR}$,$A_{if,OR,NR}$,$A_{if,OR,NR}$,$A_{if,NR,MR}$,$A_{if,NR,OR}$が算出される。

$$
\displaystyle
A_{ic,MR,OR} = A_{if,MR,NR} = \frac{ A_{if,MR}}{2}
$$
$$
\displaystyle
A_{ic,OR,MR} = A_{if,OR,NR} = \frac{ A_{if,OR}}{2}
$$
$$
\displaystyle
A_{ic,NR,MR} = A_{if,NR,OR} = \frac{ A_{if,NR}}{2}
$$

$A_{if,NR,MR}$の算出の詳細は以下。他の2用途間についても同様。

$
\displaystyle
A_{if,MR,OR} = A_{if,MR} - A_{if,MR,NR}   
$  
$
\displaystyle
　　　　= A_{if,MR} - A_{if,OR,NR} \times \frac{A_{if,MR}}{A_{if,OR}}  
$  
$
\displaystyle
　　　　= A_{if,MR} - ( A_{if,OR} - A_{if,OR,MR}) \times \frac{A_{if,MR}}{A_{if,OR}}   
$  
$
\displaystyle
　　　　= A_{if,MR} - ( A_{if,OR} - A_{if,NR,MR} \times \frac{A_{if,OR}}{A_{if,NR}}) \times \frac{A_{if,MR}}{A_{if,OR}}  
$  
$
\displaystyle
　　　　= A_{if,NR,MR}\times \frac{A_{if,MR}}{A_{if,NR}}
$  
$
\displaystyle
A_{if,MR} : A_{if,NR} = A_{if,MR,OR} : A_{if,NR,OR} = A_{if,NR,MR}\times \frac{A_{if,MR}}{A_{if,NR}} : A_{if,NR} - A_{if,NR,MR}
$

$$
\displaystyle
∴　A_{if,NR,MR} = \frac{A_{if,NR}}{2}
$$

##### Function 

In [30]:
def get_slab_area_between_rooms(d, innerwalls):
    
    def get_if(space_name, next_space_name, area):
        return {
            'name' : 'InnerFloor' + space_name + next_space_name,
            'area' : area,
            'space' : space_name,
            'nextspace' : next_space_name
        }
    
    def get_ic(space_name, next_space_name, area):
        return {
            'name' : 'InnerCeiling' + space_name + next_space_name,
            'area' : area,
            'space' : space_name,
            'nextspace' : next_space_name
        }
    
    # 室上の床面積の取得
    a_if_mr, a_if_or, a_if_nr = get_floor_area_over_room(d, innerwalls)
    
    # 室下の天井面積の取得
    a_ic_mr, a_ic_or, a_ic_nr = get_ceiling_area_below_room(d)

    # 部屋間の床面積・天井面積を求める。
    ifs = []
    ics = []
    
    if a_if_mr > 0.0 and a_if_or > 0.0:

        ifs.append(get_if('main', 'other', a_if_mr/2))
        ics.append(get_ic('other', 'main', a_if_mr/2))
        
        ifs.append(get_if('other', 'main', a_if_or/2))
        ics.append(get_ic('main', 'other', a_if_or/2))
        
    if a_if_mr > 0.0 and a_if_nr > 0.0:
        
        ifs.append(get_if('main', 'nonliving', a_if_mr/2))
        ifs.append(get_ic('nonliving', 'main', a_if_mr/2))
        
        ifs.append(get_if('nonliving', 'main', a_if_nr/2))
        ifs.append(get_ic('main', 'nonliving', a_if_nr/2))
        
    if a_if_or > 0.0 and a_if_nr > 0.0:
        
        ifs.append(get_if('other', 'nonliving', a_if_or/2))
        ifs.append(get_ic('nonliving', 'other', a_if_or/2))

        ifs.append(get_if('nonliving', 'other', a_if_nr/2))
        ifs.append(get_ic('other', 'nonliving', a_if_nr/2))
  
    return {'InnerFloors' : ifs, 'InnerCeilings' : ics}

#### 5) 室間の床・天井の構築

##### Function

In [31]:
def get_innerwall_innerslab(d, innerwalls):
    
    # 室間の床・天井面積の推定
    areas_innerslab = get_slab_area_between_rooms(d, innerwalls)
    
    # 壁体名称、部位面積を更新し、「壁体が属する空間種類」「隣接空間種類」を追加する
    ifs = []
    
    ifs.extend([ {
        'name' : 'InnerFloor_' + x['space'],
        'type' : 'InnerFloor',
        'direction' : 'Horizontal',
        'area' : x['area'],
        'Layers' : get_inner_floor_layers(),
        'space' : x['space'], 
        'nextspace' : x['nextspace'],
    } for x in areas_innerslab['InnerFloors'] ])
    
    ifs.extend([ {
        'name' : 'InnerCeiling_' + x['space'],
        'type' : 'InnerCeiling',
        'direction' : 'Horizontal',
        'area' : x['area'],
        'Layers' : get_inner_floor_layers(),
        'space' : x['space'], 
        'nextspace' : x['nextspace'],
    } for x in areas_innerslab['InnerCeilings'] ])
        
    return ifs

#### Example

In [32]:
d = {
    'Common': {
        'Region': 6,
        'IsSimplifiedInput': False,
        'MainOccupantRoomFloorArea': 30.0,
        'OtherOccupantRoomFloorArea': 60.0,
        'TotalFloorArea': 120.0
    },
    'Walls': [
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'wood', 'InputMethod' :'InputUA', 
          'direction': 'top', 'area': 60, 'space': 'main', 'UA': 0.24, 'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'wood', 'InputMethod' :'InputAllDetails', 'direction': 'top', 
          'area': 60, 'space': 'main',
          'Parts': [{'AreaRatio': 0.8, 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                  {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'AreaRatio': 0.2, 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                  {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
          'IsSunshadeInput': False },
        { 'name': 'Floor', 'type': 'Floor', 'structure': 'wood', 'InputMethod' :'InputAllLayers', 'direction': 'bottom', 
          'area': 60, 'space': 'main',
          'FloorConstructionMethod' :'FrameInsulcolumn',
          'Parts': [{'TypeFloor': 'Insulation', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                           {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'TypeFloor': 'Heatbridge', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                           {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
           'IsSunshadeInput': False }
    ],
    'Windows': [
        { 'name': 'WindowTop', 'direction': 'top', 'area': 30, 'space': 'main', 'UW': 6.51, 'TypeWindow': 'Single', 
          'IsEtaValueInput': False, 'TypeFrame': 'WoodOrResin', 'TypeGlass': '3WgG', 'TypeShade': 'Shoji', 'IsSunshadeInput': False }
    ],
    'Earthfloors': [
        { 'name': 'other', 'area': 5.0, 'space': 'underfloor' },
        { 'name': 'entrance', 'area': 5.0, 'space': 'underfloor' }
    ]
}

innerwalls = [
        { 'area': 60, 'name': 'GroundFloor_Main', 'type' : 'GroundFloor', 'space': 'main', 'nextspace' : 'underfloor', 'direction' : 'Horizontal', 'Layers' : get_inner_floor_layers()},
        { 'area': 15, 'name': 'GroundFloor_Other', 'type' : 'GroundFloor', 'space': 'other', 'nextspace' : 'underfloor', 'direction' : 'Horizontal', 'Layers' : get_inner_floor_layers()},
]

get_slab_area_between_rooms(d, innerwalls)

{'InnerFloors': [{'name': 'InnerFloorothernonliving',
   'area': 22.5,
   'space': 'other',
   'nextspace': 'nonliving'},
  {'name': 'InnerCeilingnonlivingother',
   'area': 22.5,
   'space': 'nonliving',
   'nextspace': 'other'},
  {'name': 'InnerFloornonlivingother',
   'area': 15.0,
   'space': 'nonliving',
   'nextspace': 'other'},
  {'name': 'InnerCeilingothernonliving',
   'area': 15.0,
   'space': 'other',
   'nextspace': 'nonliving'}],
 'InnerCeilings': []}

### 8.5. InnerWalls Element - Inner Wall 

#### 1) 層構成

以下の層構成とする。

| 材料 | 厚さ, m | 熱伝導率, W m<sup>-1</sup> K<sup>-1</sup> | 熱抵抗, m<sup>2</sup>2 k W<sup>-1</sup> | 容積比熱, J L<sup>-1</sup> K<sup>-1</sup> |
|---|---|---|---|
| せっこうボード | 0.0125 | 0.22 | 0.0568 | 830.0 |
| 非密閉空気層 | - | - | 0.090 | 0.0 |
| せっこうボード | 0.0125 | 0.22 | 0.0568 | 830.0 |

##### Function

In [33]:
def get_inner_wall_layers():
    return [
        {'name': 'GPB', 'cond': 0.22,   'thick': 0.0125, 'specH': 830.0},
        {'name': 'GPB', 'cond': 1/0.09, 'thick': 1,      'specH': 0},
        {'name': 'GPB', 'cond': 0.22,   'thick': 0.0125, 'specH': 830.0}
    ]

#### 2) 内壁等面積の推定

用途間の内壁等の面積は、各用途の内壁等の面積より決定する。

$$
A_{iw,MR-OR} =\frac{ A_{iw,MR} + A_{iw,OR} - A_{iw,NR} }{2}
$$

$$
A_{iw,MR-NR} = \frac{ A_{iw,MR} + A_{iw,NR} - A_{iw,OR} }{2}
$$

$$
A_{iw,NR-OR} = \frac{ A_{iw,NR} + A_{iw,OR} - A_{iw,MR} }{2}
$$

$ \because $

$$
A_{iw,MR} =A_{iw,MR-OR}+A_{iw,MR-NR}
$$

$$
A_{iw,OR} =A_{iw,MR-OR}+A_{iw,OR-NR}
$$

$$
A_{iw,NR} =A_{iw,MR-OR}+A_{iw,OR-NR}
$$

$A_{iw,MR-OR}$: 主たる居室とその他の居室の間の内壁等の面積, m<sup>2</sup>  
$A_{iw,MR-NR}$: 主たる居室と非居室の間の内壁等の面積, m<sup>2</sup>  
$A_{iw,OR-NR}$: その他の居室と非居室の間の内壁等の面積, m<sup>2</sup>  
$A_{iw,MR}$: 主たる居室の内壁等の面積, m<sup>2</sup>  
$A_{iw,OR}$: その他の居室の内壁等の面積, m<sup>2</sup>  
$A_{iw,NR}$: 非居室の内壁等の面積, m<sup>2</sup>  

各用途の内壁等の面積は、各用途の内壁等の面積の暫定値より、用途間の内壁等の面積が0㎡以上となる範囲で決定する。

$A_{iw,MR,temp}>A_{iw,OR,temp}+A_{iw,NR,temp}$ の場合

　　$A_{OR}=0$ かつ $A_{NR}=0$ の場合

$$A_{iw,MR} = 0, A_{iw,OR} = 0, A_{iw,NR} = 0$$

　　$A_{OR}>0$ または $A_{NR}>0$ で、 $A_{iw,OR,temp}=0$ かつ $A_{iw,NR,temp}=0$ の場合

$$
A_{iw,MR} = A_{iw,MR,temp}
$$
$$
A_{iw,OR} = A_{iw,MR,temp}\times \frac{A_{OR}}{A_{OR}+A_{NR}}
$$
$$
A_{iw,NR} = A_{iw,MR,temp}\times \frac{A_{NR}}{A_{OR}+A_{NR}}
$$

　　$A_{OR}>0$ または $A_{NR}>0$ で、 $A_{iw,OR,temp}>0$ または $A_{iw,NR,temp}>0$ の場合

$$
A_{iw,MR} = A_{iw,MR,temp}
$$
$$
A_{iw,OR} = A_{iw,MR,temp}\times \frac{A_{iw,OR,temp}}{A_{iw,OR,temp}+A_{iw,NR,temp}}
$$
$$
A_{iw,NR} = A_{iw,MR,temp}\times \frac{A_{iw,NR,temp}}{A_{iw,OR,temp}+A_{iw,NR,temp}}
$$

$A_{iw,OR,temp}>A_{iw,MR,temp}+A_{iw,NR,temp}$ の場合

　　$A_{MR}=0$ かつ $A_{NR}=0$ の場合

$$A_{iw,MR} = 0, A_{iw,OR} = 0, A_{iw,NR} = 0$$

　　$A_{OR}>0$ または $A_{NR}>0$ で、 $A_{iw,OR,temp}=0$ かつ $A_{iw,NR,temp}=0$ の場合

$$
A_{iw,OR} = A_{iw,OR,temp}
$$
$$
A_{iw,MR} = A_{iw,OR,temp}\times \frac{A_{MR}}{A_{MR}+A_{NR}}
$$
$$
A_{iw,NR} = A_{iw,OR,temp}\times \frac{A_{NR}}{A_{MR}+A_{NR}}
$$

　　$A_{OR}>0$ または $A_{NR}>0$ で、 $A_{iw,OR,temp}>0$ または $A_{iw,NR,temp}>0$ の場合

$$
A_{iw,OR} = A_{iw,OR,temp}
$$
$$
A_{iw,MR} = A_{iw,OR,temp}\times \frac{A_{iw,MR,temp}}{A_{iw,MR,temp}+A_{iw,NR,temp}}
$$
$$
A_{iw,NR} = A_{iw,OR,temp}\times \frac{A_{iw,NR,temp}}{A_{iw,MR,temp}+A_{iw,NR,temp}}
$$

$A_{iw,NR,temp}>A_{iw,MR,temp}+A_{iw,OR,temp}$ の場合

　　$A_{MR}=0$ かつ $A_{OR}=0$ の場合

$$A_{iw,MR} = 0, A_{iw,OR} = 0, A_{iw,NR} = 0$$

　　$A_{OR}>0$ または $A_{NR}>0$ で、 $A_{iw,OR,temp}=0$ かつ $A_{iw,NR,temp}=0$ の場合

$$
A_{iw,NR} = A_{iw,NR,temp}
$$
$$
A_{iw,MR} = A_{iw,NR,temp}\times \frac{A_{MR}}{A_{MR}+A_{OR}}
$$
$$
A_{iw,OR} = A_{iw,NR,temp}\times \frac{A_{OR}}{A_{MR}+A_{OR}}
$$

　　$A_{OR}>0$ または $A_{NR}>0$ で、 $A_{iw,OR,temp}>0$ または $A_{iw,NR,temp}>0$ の場合

$$
A_{iw,NR} = A_{iw,NR,temp}
$$
$$
A_{iw,MR} = A_{iw,NR,temp}\times \frac{A_{iw,MR,temp}}{A_{iw,MR,temp}+A_{iw,OR,temp}}
$$
$$
A_{iw,OR} = A_{iw,NR,temp}\times \frac{A_{iw,OR,temp}}{A_{iw,MR,temp}+A_{iw,OR,temp}}
$$

上記以外の場合

$$A_{iw,MR} = A_{iw,MR,temp}, A_{iw,OR} = A_{iw,OR,temp}, A_{iw,NR} = A_{iw,NR,temp}$$

$A_{MR}$: 主たる居室の床面積, m<sup>2</sup>  
$A_{OR}$: その他の居室の床面積, m<sup>2</sup>  
$A_{NR}$: 非居室の床面積, m<sup>2</sup>   
$A_{iw,MR,temp}$: 主たる居室の内壁等の面積の暫定値, m<sup>2</sup>  
$A_{iw,OR,temp}$: その他の居室の内壁等の面積の暫定値, m<sup>2</sup>  
$A_{iw,NR,temp}$: 非居室の内壁等の面積の暫定値, m<sup>2</sup>  

各用途の内壁等の面積の暫定値は、室の周長を求める際の係数（アスペクト比）と階高を仮定して、外皮の用途別情報および用途別床面積より推定する。

$$
A_{iw,MR,temp} = \left \{
\begin{array}{ll}
0 & ( A_{MR} = 0 ) \\
A_{w,MR} - A_{ow,MR} = \max \left ( 0,4 \cdot ar_{MR} \cdot h \sqrt {A_{MR}} - A_{ow,MR} \right ) & ( A_{MR} > 0 )
\end{array}
\right.
$$

$$
A_{iw,OR,temp} = \left \{
\begin{array}{ll}
0 & ( A_{OR} = 0 ) \\
A_{w,OR} - A_{ow,OR} = \max \left ( 0, 4 \cdot ar_{NR} \cdot h \sqrt {A_{OR}} - A_{ow,OR} \right ) & ( A_{OR} > 0 )
\end{array}
\right.
$$

$$
A_{iw,NR,temp} = \left \{
\begin{array}{ll}
0 & ( A_{NR} = 0 ) \\
A_{w,NR} - A_{ow,NR} = \max \left ( 0, 4 \cdot ar_{OR} \cdot h \sqrt {A_{NR}} - A_{ow,NR} \right ) & ( A_{NR} > 0 )
\end{array}
\right.
$$

$A_{w,MR}$: 主たる居室の鉛直面の外皮と内壁の面積の合計, m<sup>2</sup>  
$A_{w,OR}$: その他の居室の鉛直面の外皮と内壁の面積の合計, m<sup>2</sup>  
$A_{w,NR}$: 非居室の鉛直面の外皮と内壁の面積の合計, m<sup>2</sup>  
$A_{ow,MR}$: 主たる居室の鉛直面の外皮の面積の合計, m<sup>2</sup>  
$A_{ow,OR}$: その他の居室の鉛直面の外皮の面積の合計, m<sup>2</sup>  
$A_{ow,NR}$: 非居室の鉛直面の外皮の面積の合計, m<sup>2</sup>  
$ar_{MR}$: 主たる居室の周長を求める際の係数, -  
$ar_{OR}$: その他の居室の周長を求める際の係数, -  
$ar_{NR}$: 非居室の周長を求める際の係数, -  
$h$: 階高, m

##### Function

In [34]:
def get_area_inner_wall(d):
    
    # 用途別床面積の取得
    a_mr, a_or, a_nr = get_floor_area(a_mr=d['Common']['MainOccupantRoomFloorArea'],
                                      a_or=d['Common']['OtherOccupantRoomFloorArea'],
                                      a_total=d['Common']['TotalFloorArea'])
 
    # 周長を求める際の係数, -
    r_mr, r_or, r_nr = 1.2, 1.4, 1.4

    # 居室の外皮と床下空間との間の床の面積の合計
    a_ow_mr, a_ow_or, a_ow_nr = get_outside_wall_area(d)

    # 内壁等の面積の暫定値の取得
    def get_int_wall_area(floor_area, ratio, outside_wall_area):
        h = 2.8 # 階高, m
        return max(0.0, 4 * h * ratio * floor_area**0.5 - outside_wall_area) if floor_area > 0.0 else 0.0

    # 内壁等の面積の暫定値の取得
    a_iw_mr = get_int_wall_area(a_mr, r_mr, a_ow_mr)
    a_iw_or = get_int_wall_area(a_or, r_or, a_ow_or)
    a_iw_nr = get_int_wall_area(a_nr, r_nr, a_ow_nr)

    # 内壁等の面積の取得
    def get_alt_area(a_iw, a_iw_adj1, a_iw_adj2, a_f_adj1, a_f_adj2):
        # a_iw: 当該ZONEの内壁面積
        # a_iw_adj1, a_f_adj1: 隣のZONE1の内壁面積, 床面積
        # a_iw_adj2, a_f_adj2: 隣のZONE2の内壁面積, 床面積
        
        if a_f_adj1 == 0.0 and a_f_adj2 == 0.0:
            return 0.0, 0.0, 0.0
        elif a_iw_adj1 == 0.0 and a_iw_adj2 == 0.0:
            return a_iw, a_iw * a_f_adj1 / (a_f_adj1 + a_f_adj2), a_iw * a_f_adj2 / (a_f_adj1 + a_f_adj2)
        else:
            return a_iw, a_iw * a_iw_adj1 / (a_iw_adj1 + a_iw_adj2), a_iw * a_iw_adj2 / (a_iw_adj1 + a_iw_adj2)

    if a_iw_mr > a_iw_or + a_iw_nr:
        alt_a_iw_mr, alt_a_iw_or, alt_a_iw_nr = get_alt_area(a_iw_mr, a_iw_or, a_iw_nr, a_or, a_nr)
    elif a_iw_or > a_iw_nr + a_iw_mr:
        alt_a_iw_or, alt_a_iw_nr, alt_a_iw_mr = get_alt_area(a_iw_or, a_iw_nr, a_iw_mr, a_nr, a_mr)
    elif a_iw_nr > a_iw_mr + a_iw_or:
        alt_a_iw_nr, alt_a_iw_mr, alt_a_iw_or = get_alt_area(a_iw_nr, a_iw_mr, a_iw_or, a_mr, a_or)
    else :
        alt_a_iw_mr, alt_a_iw_or, alt_a_iw_nr = a_iw_mr, a_iw_or, a_iw_nr
    
    # 室用途間の内壁等の面積の推定
    a_iw_mr_or = (alt_a_iw_mr + alt_a_iw_or - alt_a_iw_nr) / 2
    a_iw_mr_nr = (alt_a_iw_nr + alt_a_iw_mr - alt_a_iw_or) / 2
    a_iw_or_nr = (alt_a_iw_or + alt_a_iw_nr - alt_a_iw_mr) / 2
    
    return a_iw_mr_or, a_iw_mr_nr, a_iw_or_nr           

#### 3) 内壁の構築

##### Function

In [35]:
def get_innerwall_wall(d):

    # 内壁面積の推定
    a_iw_mr_or, a_iw_mr_nr, a_iw_or_nr = get_area_inner_wall(d)
    
    # 壁体名称、部位面積を更新し、「壁体が属する空間種類」「隣接空間種類」を追加する
    iws = []

    if a_iw_mr_or > 0.0:

        iws.append({
            'name' : 'Innerwall_main to other',
            'type' : 'InnerWall',
            'direction' : 'Vertical',
            'Layers' : get_inner_wall_layers(),
            'area' : a_iw_mr_or,
            'space' : 'main', 
            'nextspace:': 'other'
        })
        
        iws.append({
            'name' : 'Innerwall_main to other',
            'type' : 'InnerWall',
            'direction' : 'Vertical',
            'Layers' : get_inner_wall_layers(),
            'area' : a_iw_mr_or,
            'space' : 'other', 
            'nextspace:': 'main'
        })

    if a_iw_mr_nr > 0.0:
        
        iws.append({
            'name' : 'Innerwall_main to nonliving',
            'type' : 'InnerWall',
            'direction' : 'Vertical',
            'Layers' : get_inner_wall_layers(),
            'area' : a_iw_mr_nr,
            'space' : 'main', 
            'nextspace:': 'nonliving'
        })
        
        iws.append({
            'name' : 'Innerwall_main to nonliving',
            'type' : 'InnerWall',
            'direction' : 'Vertical',
            'Layers' : get_inner_wall_layers(),
            'area' : a_iw_mr_nr,
            'space' : 'nonliving', 
            'nextspace:': 'main'
        })

    if a_iw_or_nr > 0.0:
        
        iws.append({
            'name' : 'Innerwall_other to nonliving',
            'type' : 'InnerWall',
            'direction' : 'Vertical',
            'Layers' : get_inner_wall_layers(),
            'area' : a_iw_or_nr,
            'space' : 'other', 
            'nextspace:': 'nonliving'
        })
        iws.append({
            'name' : 'Innerwall_other to nonliving',
            'type' : 'InnerWall',
            'direction' : 'Vertical',
            'Layers' : get_inner_wall_layers(),
            'area' : a_iw_or_nr,
            'space' : 'nonliving', 
            'nextspace:': 'other'
        })

    return iws

##### Example

In [36]:
correspondence = {
    'main to other': ['main', 'other'],
    'main to nonliving': ['main', 'nonliving'],
    'other to nonliving': ['other', 'nonliving'] 
}  

d = {
    'Common': {
        'Region': 6,
        'IsSimplifiedInput': False,
        'MainOccupantRoomFloorArea': 30.0,
        'OtherOccupantRoomFloorArea': 60.0,
        'TotalFloorArea': 120.0
    },
    'Walls': [
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'wood', 'InputMethod' :'InputUA', 
          'direction': 'top', 'area': 60, 'space': 'main', 'UA': 0.24, 'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'wood', 'InputMethod' :'InputAllDetails', 'direction': 'top', 
          'IsInContactWithOutsideAir': True, 'area': 60, 'space': 'main',
          'Parts': [{'AreaRatio': 0.8, 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                  {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'AreaRatio': 0.2, 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                  {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
          'IsSunshadeInput': False },
        { 'name': 'Floor', 'type': 'Floor', 'structure': 'wood', 'InputMethod' :'InputAllLayers', 'direction': 'bottom', 
          'IsInContactWithOutsideAir': True, 'area': 60, 'space': 'main',
          'FloorConstructionMethod' :'FrameInsulcolumn',
          'Parts': [{'TypeFloor': 'Insulation', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                           {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'TypeFloor': 'Heatbridge', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                           {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
           'IsSunshadeInput': False }
    ],
    'Windows': [
        { 'name': 'WindowTop', 'direction': 'top', 'area': 30, 'space': 'main', 'UW': 6.51, 'TypeWindow': 'Single', 
          'IsEtaValueInput': False, 'TypeFrame': 'WoodOrResin', 'TypeGlass': '3WgG', 'TypeShade': 'Shoji', 'IsSunshadeInput': False }
    ],
    'Earthfloors': [
        { 'name': 'other', 'area': 5.0, 'space': 'underfloor' },
        { 'name': 'entrance', 'area': 5.0, 'space': 'underfloor' }
    ]
}

innerwalls = [
        { 'area': 60, 'name': 'GroundFloor_Main', 'type' : 'GroundFloor', 'spaces': ['main','underfloor'], 'direction' : 'Horizontal', 'Layers' : get_inner_floor_layers()},
        { 'area': 15, 'name': 'GroundFloor_Other', 'type' : 'GroundFloor', 'spaces': ['other','underfloor'], 'direction' : 'Horizontal', 'Layers' : get_inner_floor_layers()},
]

get_innerwall_wall(d)

[{'name': 'Innerwall_main to other',
  'type': 'InnerWall',
  'direction': 'Vertical',
  'Layers': [{'name': 'GPB', 'cond': 0.22, 'thick': 0.0125, 'specH': 830.0},
   {'name': 'GPB', 'cond': 11.11111111111111, 'thick': 1, 'specH': 0},
   {'name': 'GPB', 'cond': 0.22, 'thick': 0.0125, 'specH': 830.0}],
  'area': 54.59388622447443,
  'space': 'main',
  'nextspace:': 'other'},
 {'name': 'Innerwall_main to other',
  'type': 'InnerWall',
  'direction': 'Vertical',
  'Layers': [{'name': 'GPB', 'cond': 0.22, 'thick': 0.0125, 'specH': 830.0},
   {'name': 'GPB', 'cond': 11.11111111111111, 'thick': 1, 'specH': 0},
   {'name': 'GPB', 'cond': 0.22, 'thick': 0.0125, 'specH': 830.0}],
  'area': 54.59388622447443,
  'space': 'other',
  'nextspace:': 'main'},
 {'name': 'Innerwall_main to nonliving',
  'type': 'InnerWall',
  'direction': 'Vertical',
  'Layers': [{'name': 'GPB', 'cond': 0.22, 'thick': 0.0125, 'specH': 830.0},
   {'name': 'GPB', 'cond': 11.11111111111111, 'thick': 1, 'specH': 0},
   {'na

### 8.3. InnerWalls Element - 統合

##### Function

In [37]:
def convert_innerwall(d):

    # 室と床下空間の間の床の推定
    iws_to_groundfloor = get_innerwall_groundfloor(d)
    
    # 室間の床・天井の推定
    iws_slab = get_innerwall_innerslab(d, iws_to_groundfloor)
    
    # 内壁等の推定              
    iws_wall = get_innerwall_wall(d)
    
    return iws_to_groundfloor + iws_slab + iws_wall

##### Example

In [38]:
d = {
    'Common': {
        'Region': 6,
        'IsSimplifiedInput': False,
        'MainOccupantRoomFloorArea': 30.0,
        'OtherOccupantRoomFloorArea': 60.0,
        'TotalFloorArea': 120.0
    },
    'Walls': [
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'wood', 'InputMethod' :'InputUA', 
          'direction': 'top', 'area': 60, 'space': 'main', 'UA': 0.24, 'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'wood', 'InputMethod' :'InputAllDetails', 'direction': 'top', 
          'IsInContactWithOutsideAir': True, 'area': 60, 'space': 'main',
          'Parts': [{'AreaRatio': 0.8, 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                  {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'AreaRatio': 0.2, 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                  {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
          'IsSunshadeInput': False },
        { 'name': 'Floor', 'type': 'Floor', 'structure': 'wood', 'InputMethod' :'InputAllLayers', 'direction': 'bottom', 
          'IsInContactWithOutsideAir': True, 'area': 60, 'space': 'main',
          'FloorConstructionMethod' :'FrameInsulcolumn',
          'Parts': [{'TypeFloor': 'Insulation', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                           {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'TypeFloor': 'Heatbridge', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                           {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
           'IsSunshadeInput': False }
    ],
    'Windows': [
        { 'name': 'WindowTop', 'direction': 'top', 'area': 30, 'space': 'main', 'UW': 6.51, 'TypeWindow': 'Single', 
          'IsEtaValueInput': False, 'TypeFrame': 'WoodOrResin', 'TypeGlass': '3WgG', 'TypeShade': 'Shoji', 'IsSunshadeInput': False }
    ],
    'Earthfloors': [
        { 'name': 'other', 'area': 5.0, 'space': 'underfloor' },
        { 'name': 'entrance', 'area': 5.0, 'space': 'underfloor' }
    ]
}

innerwalls = [
        { 'area': 60, 'name': 'GroundFloor_Main', 'type' : 'GroundFloor', 'spaces': ['main','underfloor'], 'direction' : 'Horizontal', 'Layers' : get_inner_floor_layers()},
        { 'area': 15, 'name': 'GroundFloor_Other', 'type' : 'GroundFloor', 'spaces': ['other','underfloor'], 'direction' : 'Horizontal', 'Layers' : get_inner_floor_layers()},
]

convert_innerwall( d )

[{'name': 'GroundFloor_Other',
  'type': 'GroundFloor',
  'direction': 'Horizontal',
  'area': 6.666666666666667,
  'Layers': [{'name': 'PED', 'cond': 0.16, 'thick': 0.012, 'specH': 720.0}],
  'space': 'other',
  'nextspace': 'underfloor'},
 {'name': 'GroundFloor_Other',
  'type': 'GroundFloor',
  'direction': 'Horizontal',
  'area': 6.666666666666667,
  'Layers': [{'name': 'PED', 'cond': 0.16, 'thick': 0.012, 'specH': 720.0}],
  'space': 'underfloor',
  'nextspace': 'other'},
 {'name': 'GroundFloor_Nonliving',
  'type': 'GroundFloor',
  'direction': 'Horizontal',
  'area': 3.3333333333333335,
  'Layers': [{'name': 'PED', 'cond': 0.16, 'thick': 0.012, 'specH': 720.0}],
  'space': 'nonliving',
  'nextspace': 'underfloor'},
 {'name': 'GroundFloor_Nonliving',
  'type': 'GroundFloor',
  'direction': 'Horizontal',
  'area': 3.3333333333333335,
  'Layers': [{'name': 'PED', 'cond': 0.16, 'thick': 0.012, 'specH': 720.0}],
  'space': 'underfloor',
  'nextspace': 'nonliving'},
 {'name': 'InnerFl

## 9. Lv4入力情報のコンバート（統合）
- 1)共通情報、2)開口部以外の不透明部位、3)開口部の透明部位、4)開口部の不透明部位、5) 線熱橋、6)土間床等の外周部の線熱橋、7)土間床、8)内壁等の各コンバート関数を統合し、Lv3の入力情報をLv4の入力情報にコンバートする。

##### Function

In [39]:
def convert(d):
    d_lv4_input = {}
    d_lv4_input['Common'] = convert_common(d)
    if ('Walls' in d) == True:
        d_lv4_input['Walls'] = convert_wall(d)
    if ('Windows' in d) == True:
        d_lv4_input['Windows'] = convert_window(d)
    if ('Doors' in d) == True:
        d_lv4_input['Doors'] = convert_door(d)
    if ('EarthfloorPerimeters' in d) == True:
        d_lv4_input['EarthfloorPerimeters'] = convert_earthfloorperimeter(d)
    if ('EarthFloors' in d) == True:
        d_lv4_input['EarthFloors'] = convert_earthfloor(d)
    if ('Heatbridges' in d) == True:
        d_lv4_input['Heatbridges'] = convert_heatbridge(d)
    d_lv4_input['InnerWalls'] = convert_innerwall(d)

    return d_lv4_input

##### Example

In [40]:
d = {
    'Common': {
        'Region': 6,
        'IsSimplifiedInput': False,
        'MainOccupantRoomFloorArea': 30.0,
        'OtherOccupantRoomFloorArea': 60.0,
        'TotalFloorArea': 120.0
    },
    'Walls': [
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'wood', 'InputMethod' :'InputUA', 
          'direction': 'top', 'area': 67.8, 'space': 'main', 'UA': 0.24, 'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'wood', 'InputMethod' :'InputAllDetails', 'direction': 'top', 
          'IsInContactWithOutsideAir': True, 'area': 67.8, 'space': 'main',
          'Parts': [{'AreaRatio': 0.8, 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                  {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'AreaRatio': 0.2, 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                  {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
          'IsSunshadeInput': False },
        { 'name': 'Floor', 'type': 'Floor', 'space': 'main', 'structure': 'wood', 'InputMethod' :'InputAllLayers', 'direction': 'bottom', 
          'IsInContactWithOutsideAir': True, 'area': 67.8,
          'ConstructionMethodAllLayers' :'FloorFrameBeamInsuljoist',
          'Parts': [{'Type': 'FloorInsulation', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                           {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'Type': 'FloorHeatbridge', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                           {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
           'IsSunshadeInput': False },
        { 'name': 'Wall', 'type': 'Wall', 'space': 'main', 'structure': 'wood', 'InputMethod' :'InputAllLayers', 'direction': 'N',
          'IsInContactWithOutsideAir': True, 'area': 67.8, 'ConstructionMethodAllLayers': 'WallFrameInsulcolumn',
          'Parts': [{'Type': 'WallInsulation', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                          {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'Type': 'WallHeatbridge', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                          {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],         
          'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'Roof', 'space': 'main', 'structure': 'wood', 'InputMethod' :'InputAllLayers', 'direction': 'top',
          'IsInContactWithOutsideAir': True, 'area': 67.8, 'ConstructionMethodAllLayers': 'RoofInsulRafter', 
          'Parts': [{'Type': 'RoofInsulation', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                          {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'Type': 'RoofHeatbridge', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                          {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
          'IsSunshadeInput': False },
        { 'name': 'BoundaryCeiling', 'type': 'BoundaryCeiling', 'space': 'main', 'structure': 'wood', 'InputMethod' :'InputAllLayers', 
          'direction': 'top', 'IsInContactWithOutsideAir': True, 'area': 67.8, 'ConstructionMethodAllLayers': 'CeilingInsulbeam', 
          'Parts': [{'Type': 'CeilingInsulation', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                             {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'Type': 'CeilingHeatbridge', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                             {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
          'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'wood', 'InputMethod' :'InputUR', 'InsulationMethod': 'CeilingInsulbeam',
          'direction': 'top', 'IsInContactWithOutsideAir': True, 'area': 67.8, 'space': 'main', 
          'Parts': [{ 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                 {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
          'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'RC', 'InputMethod' :'InputUA', 
          'direction': 'top','area': 67.8, 'space': 'main', 'UA': 0.24, 'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'RC', 'InputMethod' :'InputLayers', 
          'direction': 'top', 'IsInContactWithOutsideAir': True, 'area': 67.8, 'space': 'main',
          'Parts': [{ 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                 {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],         
          'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'steel', 'InputMethod' :'InputUA', 
          'direction': 'top', 'area': 67.8, 'space': 'main', 'UA': 0.24, 'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'steel', 'InputMethod' :'InputUR', 
          'direction': 'top', 'IsInContactWithOutsideAir': True, 'area': 67.8, 'space': 'main', 'Radding': 0.10, 
          'Parts': [{'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],         
          'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'other', 'direction': 'top', 'IsInContactWithOutsideAir': True, 
          'area': 67.8, 'space': 'main', 'IsSunshadeInput': False }
    ],
    'Windows': [
        { 'name': 'WindowSW', 'direction': 'SW', 'area': 30.25, 'space': 'main', 'UW': 6.51, 'TypeWindow': 'Single', 
          'IsEtaValueInput': False, 'TypeFrame': 'WoodOrResin', 'TypeGlass': '3WgG', 'TypeShade': 'Shoji',
          'IsSunshadeInput': True, 'Y1': 0.00, 'Y2': 1.00, 'Z': 0.60 },
        { 'name': 'WindowNW', 'direction': 'NW', 'area': 3.17, 'space': 'main', 'UW': 4.65, 'TypeWindow': 'Single', 
          'IsEtaValueInput': True, 'Eta': 0.738, 'IsSunshadeInput': False },
        { 'name': 'WindowSW', 'direction': 'SW', 'area': 30.25, 'space': 'main', 'UW': 6.51, 'TypeWindow': 'Double', 'IsEtaValueInput': False,
          'TypeFrameInside': 'WoodOrResin', 'TypeGlassInside': '3WgG', 'TypeShadeInside': 'Shoji',
          'TypeFrameOutside': 'Steel', 'TypeGlassOutside': '3WgG', 'TypeShadeOutside': 'ExtarnalBlind',
          'IsSunshadeInput': True, 'Y1': 0.00, 'Y2': 1.00, 'Z': 0.60 },
        { 'name': 'WindowNW', 'direction': 'NW', 'area': 3.17, 'space': 'main', 'UW': 4.65, 'TypeWindow': 'Double', 'IsEtaValueInput': True,
          'EtaInside': 0.738, 'TypeGlassInside': '3WgG', 'EtaOutside': 0.738, 'TypeGlassOutside': '3WgG',
          'IsSunshadeInput': True, 'Y1': 0.00, 'Y2': 1.00, 'Z': 0.60 }
    ],
    'Doors': [
        { 'name': 'DoorNW', 'direction': 'NW', 'area': 2.52, 'U': 6.51, 'space': 'main', 'IsSunshadeInput': False },
        { 'name': 'DoorNE', 'direction': 'NE', 'area': 2.16, 'U': 4.65, 'space': 'main', 
          'IsSunshadeInput': True, 'Y1': 0.00, 'Y2': 1.00, 'Z': 0.60 }
    ],
    'Heatbridges': [
        { 'name': 'NE', 'structure': 'RC', 'length': 1.00, 'psi': 1.8, 'direction1': 'N', 'direction2': 'E', 'space': 'main' },
        { 'name': 'NW', 'structure': 'Steel', 'length': 2.00, 'psi': 1.8, 'direction1': 'N', 'direction2': 'W', 'space': 'main' }
    ],
    'EarthfloorPerimeters': [
        { 'name': 'NW', 'direction': 'NW', 'length': 2.43, 'psi': 1.8, 'space': 'underfloor' },
        { 'name': 'NE', 'direction': 'NE', 'length': 2.43, 'psi': 1.8, 'space': 'underfloor' }
    ],
    'Earthfloors': [
        { 'name': 'other', 'area': 5.0, 'space': 'underfloor' },
        { 'name': 'entrance', 'area': 5.0, 'space': 'underfloor' }
    ]
}

convert(d)

{'Common': {'Region': 6,
  'IsSimplifiedInput': False,
  'MainOccupantRoomFloorArea': 30.0,
  'OtherOccupantRoomFloorArea': 60.0,
  'TotalFloorArea': 120.0},
 'Walls': [{'name': 'Ceiling',
   'type': 'Ceiling',
   'structure': 'wood',
   'InputMethod': 'InputUA',
   'direction': 'top',
   'area': 67.8,
   'space': 'main',
   'UA': 0.24,
   'IsSunshadeInput': False},
  {'name': 'Ceiling',
   'type': 'Ceiling',
   'structure': 'wood',
   'InputMethod': 'InputAllDetails',
   'direction': 'top',
   'IsInContactWithOutsideAir': True,
   'area': 67.8,
   'space': 'main',
   'Parts': [{'AreaRatio': 0.8,
     'Layers': [{'name': 'wood', 'thick': 0.012, 'cond': 0.16, 'specH': 720},
      {'name': 'wood', 'thick': 0.012, 'cond': 0.16, 'specH': 720}]},
    {'AreaRatio': 0.2,
     'Layers': [{'name': 'wood', 'thick': 0.012, 'cond': 0.16, 'specH': 720},
      {'name': 'wood', 'thick': 0.012, 'cond': 0.16, 'specH': 720}]}],
   'IsSunshadeInput': False},
  {'name': 'Floor',
   'type': 'Floor',
   'sp

In [41]:
d = {
    'Common': {
        'Region': 6,
        'IsSimplifiedInput': True,
        'MainOccupantRoomFloorArea': 30.0,
        'OtherOccupantRoomFloorArea': 30.0,
        'TotalFloorArea': 120.0
    },
    'Walls': [
        {'name': 'Ceiling_main', 'direction': 'Top', 'area': 16.95, 'space': 'main', 'type': 'Ceiling',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 7.7},
        {'name': 'Ceiling_other', 'direction': 'Top', 'area': 16.95, 'space': 'other', 'type': 'Ceiling',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 7.7},
        {'name': 'Ceiling_nonliving', 'direction': 'Top', 'area': 33.9, 'space': 'nonliving', 'type': 'Ceiling',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 7.7},
        {'name': 'Wall_SW_main', 'direction': 'SW', 'area': 10.1575, 'space': 'main', 'type': 'Wall',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 6.67},
        {'name': 'Wall_SW_other', 'direction': 'SW', 'area': 10.1575, 'space': 'other', 'type': 'Wall',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 6.67},
        {'name': 'Wall_SW_nonliving', 'direction': 'SW', 'area': 20.315, 'space': 'nonliving', 'type': 'Wall',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 6.67},
        {'name': 'Wall_NW_main', 'direction': 'NW', 'area': 7.4575, 'space': 'main', 'type': 'Wall',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 6.67},
        {'name': 'Wall_NW_other', 'direction': 'NW', 'area': 7.4575, 'space': 'other', 'type': 'Wall',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 6.67},
        {'name': 'Wall_NW_nonliving', 'direction': 'NW', 'area': 14.915, 'space': 'nonliving', 'type': 'Wall',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 6.67},
        {'name': 'Wall_NE_main', 'direction': 'NE', 'area': 15.9725, 'space': 'main', 'type': 'Wall',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 6.67},
        {'name': 'Wall_NE_other', 'direction': 'NE', 'area': 15.9725, 'space': 'other', 'type': 'Wall',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 6.67},
        {'name': 'Wall_NE_nonliving', 'direction': 'NE', 'area': 31.945, 'space': 'nonliving', 'type': 'Wall',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 6.67},
        {'name': 'Wall_SE_main', 'direction': 'SE', 'area': 7.4275, 'space': 'main', 'type': 'Wall',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 6.67},
        {'name': 'Wall_SE_other', 'direction': 'SE', 'area': 7.4275, 'space': 'other', 'type': 'Wall',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 6.67},
        {'name': 'Wall_SE_nonliving', 'direction': 'SE', 'area': 14.855, 'space': 'nonliving', 'type': 'Wall',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 6.67},
        {'name': 'Floor_main', 'direction': 'Bottom', 'area': 15.0175, 'space': 'main', 'type': 'Floor',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 5.27},
        {'name': 'Floor_other', 'direction': 'Bottom', 'area': 15.0175, 'space': 'other', 'type': 'Floor',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 5.27},
        {'name': 'Floor_nonliving', 'direction': 'Bottom', 'area': 30.035, 'space': 'nonliving', 'type': 'Floor',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 5.27},
        {'name': 'Floor_bath_main', 'direction': 'Bottom', 'area': 1.1025, 'space': 'main', 'type': 'Floor',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 5.27},
        {'name': 'Floor_bath_other', 'direction': 'Bottom', 'area': 1.1025, 'space': 'other', 'type': 'Floor',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 5.27},
        {'name': 'Floor_bath_nonliving', 'direction': 'Bottom', 'area': 2.205, 'space': 'nonliving', 'type': 'Floor',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 5.27}
    ],
    'Windows': [
        {'name': 'WindowSW_main', 'direction': 'SW', 'area': 7.5625, 'space': 'main', 'UW': 3.49,
         'IsSunshadeInput': True, 'EtaCooling': None, 'EtaHeating': None, 'TypeGlass': None, 'Y1': 0, 'Y2': 1.1, 'Z': None},
        {'name': 'WindowSW_other', 'direction': 'SW', 'area': 7.5625, 'space': 'other', 'UW': 3.49,
         'IsSunshadeInput': True, 'EtaCooling': None, 'EtaHeating': None, 'TypeGlass': None, 'Y1': 0, 'Y2': 1.1, 'Z': None},
        {'name': 'WindowSW_nonliving', 'direction': 'SW', 'area': 15.125, 'space': 'nonliving', 'UW': 3.49,
         'IsSunshadeInput': True, 'EtaCooling': None, 'EtaHeating': None, 'TypeGlass': None, 'Y1': 0, 'Y2': 1.1, 'Z': None},
        {'name': 'WindowNW_main', 'direction': 'NW', 'area': 0.7925, 'space': 'main', 'UW': 3.49,
         'IsSunshadeInput': True, 'EtaCooling': None, 'EtaHeating': None, 'TypeGlass': None, 'Y1': 0, 'Y2': 1.1, 'Z': None},
        {'name': 'WindowNW_other', 'direction': 'NW', 'area': 0.7925, 'space': 'other', 'UW': 3.49,
         'IsSunshadeInput': True, 'EtaCooling': None, 'EtaHeating': None, 'TypeGlass': None, 'Y1': 0, 'Y2': 1.1, 'Z': None},
        {'name': 'WindowNW_nonliving', 'direction': 'NW', 'area': 1.585, 'space': 'nonliving', 'UW': 3.49,
         'IsSunshadeInput': True, 'EtaCooling': None, 'EtaHeating': None, 'TypeGlass': None, 'Y1': 0, 'Y2': 1.1, 'Z': None},
        {'name': 'WindowNE_main', 'direction': 'NE', 'area': 1.21, 'space': 'main', 'UW': 3.49,
         'IsSunshadeInput': True, 'EtaCooling': None, 'EtaHeating': None, 'TypeGlass': None, 'Y1': 0, 'Y2': 1.1, 'Z': None},
        {'name': 'WindowNE_other', 'direction': 'NE', 'area': 1.21, 'space': 'other', 'UW': 3.49,
         'IsSunshadeInput': True, 'EtaCooling': None, 'EtaHeating': None, 'TypeGlass': None, 'Y1': 0, 'Y2': 1.1, 'Z': None},
        {'name': 'WindowNE_nonliving', 'direction': 'NE', 'area': 2.42, 'space': 'nonliving', 'UW': 3.49,
         'IsSunshadeInput': True, 'EtaCooling': None, 'EtaHeating': None, 'TypeGlass': None, 'Y1': 0, 'Y2': 1.1, 'Z': None},
        {'name': 'WindowSE_main', 'direction': 'SE', 'area': 1.4575, 'space': 'main', 'UW': 3.49,
         'IsSunshadeInput': True, 'EtaCooling': None, 'EtaHeating': None, 'TypeGlass': None, 'Y1': 0, 'Y2': 1.1, 'Z': None},
        {'name': 'WindowSE_other', 'direction': 'SE', 'area': 1.4575, 'space': 'other', 'UW': 3.49,
         'IsSunshadeInput': True, 'EtaCooling': None, 'EtaHeating': None, 'TypeGlass': None, 'Y1': 0, 'Y2': 1.1, 'Z': None},
        {'name': 'WindowSE_nonliving', 'direction': 'SE', 'area': 2.915, 'space': 'nonliving', 'UW': 3.49,
         'IsSunshadeInput': True, 'EtaCooling': None, 'EtaHeating': None, 'TypeGlass': None, 'Y1': 0, 'Y2': 1.1, 'Z': None}
    ],
    'Doors': [
        {'name': 'DoorNW_main', 'direction': 'NW', 'area': 0.63, 'space': 'main', 'U': 4.65, 'IsSunshadeInput': False},
        {'name': 'DoorNW_other', 'direction': 'NW', 'area': 0.63, 'space': 'other', 'U': 4.65, 'IsSunshadeInput': False},
        {'name': 'DoorNW_nonliving', 'direction': 'NW', 'area': 1.26, 'space': 'nonliving', 'U': 4.65, 'IsSunshadeInput': False},
        {'name': 'DoorNE_main', 'direction': 'NE', 'area': 0.54, 'space': 'main', 'U': 4.65, 'IsSunshadeInput': False},
        {'name': 'DoorNE_other', 'direction': 'NE', 'area': 0.54, 'space': 'other', 'U': 4.65, 'IsSunshadeInput': False},
        {'name': 'DoorNE_nonliving', 'direction': 'NE', 'area': 1.08, 'space': 'nonliving', 'U': 4.65, 'IsSunshadeInput': False}
    ],
    'EarthfloorPerimeters': [
        {'direction': 'NW', 'length': 2.43, 'name': 'Entrance_NW', 'psi': 1.8, 'space': 'underfloor'},
        {'direction': 'NE', 'length': 1.83, 'name': 'Entrance_NE', 'psi': 1.8, 'space': 'underfloor'},
        {'direction': 'OpenBackFloor', 'length': 4.25, 'name': 'Entrance_floor', 'psi': 1.8, 'space': 'underfloor'}
    ]
}

convert(d)

{'Common': {'Region': 6,
  'IsSimplifiedInput': True,
  'MainOccupantRoomFloorArea': 30.0,
  'OtherOccupantRoomFloorArea': 30.0,
  'TotalFloorArea': 120.0},
 'Walls': [{'name': 'Ceiling_main',
   'direction': 'Top',
   'area': 16.95,
   'space': 'main',
   'type': 'Ceiling',
   'structure': 'wood',
   'IsSunshadeInput': False,
   'UA': 7.7},
  {'name': 'Ceiling_other',
   'direction': 'Top',
   'area': 16.95,
   'space': 'other',
   'type': 'Ceiling',
   'structure': 'wood',
   'IsSunshadeInput': False,
   'UA': 7.7},
  {'name': 'Ceiling_nonliving',
   'direction': 'Top',
   'area': 33.9,
   'space': 'nonliving',
   'type': 'Ceiling',
   'structure': 'wood',
   'IsSunshadeInput': False,
   'UA': 7.7},
  {'name': 'Wall_SW_main',
   'direction': 'SW',
   'area': 10.1575,
   'space': 'main',
   'type': 'Wall',
   'structure': 'wood',
   'IsSunshadeInput': False,
   'UA': 6.67},
  {'name': 'Wall_SW_other',
   'direction': 'SW',
   'area': 10.1575,
   'space': 'other',
   'type': 'Wall',
  